### **1. Proceso de _ETL_ de los datos del Proyecto Individual 01**
### **(_Extract, Transformation, Loading_)** 

<br> <br> <br> 

El presente _Jupyter notebook_ contiene el código en _Python_ para el desarrollo  de la extracción, transformación y carga del conjunto de datos del Proyecto Individual 01; el cual es un paso fundamental para los procesos siguientes de _EDA_ e implementación del modelo de Machine Learning ( _ML_)

<br> <br> <br> <br>

<br> <br> <br> <br><br> <br> <br> <br>

---
---
**1.1. Importación de las bibliotecas de _Python_ que serán usadas para el proceso de _ETL_ de los datos del proyecto**

In [ ]:

import pandas as pd
import numpy as np
import json
import ast
import matplotlib

#----------------------------------------------------------------------------------------------------

from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder

#----------------------------------------------------------------------------------------------------

%load_ext autoreload
import FuncionesDA

    # El archivo FuncionesDA.py contiene funciones que permiten simplificar el
    # proceso de ETL de los datos 
#----------------------------------------------------------------------------------------------------

<br> <br> <br>
<br> <br> <br> <br><br> <br> <br> <br>

---
---

**1.2. Proceso de _ETL_ para los archivos de datos del proyecto**



Extracción de los datos desde el archivo con extensión .json, conversión en un objeto tipo _Dataframe_ (manejo de la librería _Pandas_), observación de su contenido, transformación de las variables pertinentes para el proyecto y carga de los datos tranformados a un formato apropiado.

<br> <br>

<br> <br> <br> <br><br> <br> <br> <br>
***

**1.2.1.  Archivo `output_steam_games.json`**

Extracción de los datos desde el archivo de formato _json_, conversión en Dataframe para su observación preliminar del contenido y transformación posterior de los datos.

In [ ]:

filepath_games = "Data/Raw Data/output_steam_games.json" 
  #Ruta al conjunto de datos australian_user_reviews.json 
  #Se guarda en la variable filepath_games

rows_games = []                          
with open(filepath_games) as file:
    for line in file.readlines():
         Data_01 = json.loads(line)
         rows_games.append(Data_01)    
   #Lectura de cada una de las líneas del conjunto de datos en formato JSON
   #Cada línea se almacena en el objeto rows_games

Data_Games = pd.DataFrame(rows_games)
   #Creación de un Dataframe (Pandas) con las filas almacenadas en rows_games'''

Data_Games


In [ ]:

print(f"En principio, el conjunto de datos contiene {Data_Games.shape[0]} filas y {Data_Games.shape[1]} variables")

 La observación preliminar del Dataframe _Data_Games_  muestra la existencia registros vacíos en algunas de las variables (si no en todas ellas). Se procede, por tanto, a la eliminación dichos registros.

In [ ]:
Data_Games = Data_Games.dropna(how='all').reset_index(drop=True)

Data_Games

Se determina el tamaño tanto de filas como de columnas del conjunto de datos y sus características generales


In [ ]:
Data_Games.shape, Data_Games.info


1. Presentación de la tabla detallando los tipos de datos por cada variable (columna) y la cantidad de valores nulos que hay en cada campo.

In [ ]:
FuncionesDA.Data_Type(Data_Games)

2. Revisión de algunas campos para conocer sus características

In [ ]:
# Ejemplo del  tipo de datos contenido en la variable 'tags'
Data_Games['tags'][0]

In [ ]:
# Ejemplo del  tipo de datos contenido en la variable 'specs'
Data_Games['specs'][101]

In [ ]:
# Observaciones únicas en la variable 'price'
Data_Games['price'].unique()

In [ ]:
# Observaciones contenidas en el campo 'early_access'
Data_Games['early_access'].unique()

In [ ]:
Data_Games['publisher']

In [ ]:
Data_Games['release_date']

In [ ]:
Data_Games['early_access']

A continuación aparece la reseña de cada una de las columnas del conjunto de datos:

* **publisher**: Empresa que publicó el juego.
* **genres**: Es el género al cual pertenece el juego. Esta formado por una lista de uno o mas géneros por registro.
* **app_name**: Nombre del juego.
* **title**: Título del juego.
* **url**: La _url_ (dirección web) del juego.
* **release_date**: Fecha de lanzamiento del juego. Su formato YYYY-MM-DD.
* **tags**: Etiqueta del juego. Esta formado por una lista con una o más etiquetas por cada registro.
* **reviews_url**: La _url_ donde se encuentra la reseña del juego.
* **specs**: Especificaciones de cada juego. Viene en formato lista.
* **price**: Precio del juego.
* **early_access**: Acceso temprano. Formato binario. 
* **id**: Identificador único del juego.
* **developer**: Nombre del desarrollador del juego.

Este conjunto contiene 32135 filas y 13 columnas. Aún existen registros nulos (_NaN_) en ciertas columnas. 

<br><br>

3. Verificación de observaciones duplicadas,teniendo en cuenta la columna del **id** del juego.

In [ ]:
FuncionesDA.Duplicates(Data_Games, "id")

Solamente hay dos items duplicados correspondiente al **id** 612880, por lo que se puede eliminar cualquier de ellos. Por otra parte se observa que la segunda dupla de duplicados el **id**  correspondiente es nulo y la primera aparición presenta mas cantidad de _NaN_ que la segunda aparición.

Se busca por la columna **developer** si este juego ya se encuentra registrado.

In [ ]:
Data_Games[Data_Games['developer']=='Rocksteady Studios,Feral Interactive (Mac)']

En efecto,  el juego que no tiene sin identificador corresponde al **id** 200260 y que el registro se encuentra completo. Por lo que se pueden borrar ambas filas dupliacadas.

In [ ]:
# Eliminación de los indices
suprimir_indices = [14573, 74, 30961]
Data_Games = Data_Games.drop(suprimir_indices)

# Verificación adicional de registros duplicados
FuncionesDA.Duplicates(Data_Games, 'id')

4. Transformación de la columna **release_date**.

 Para el trabajo posterior del proyecto, resulta necesario extraer el año de lanzamiento del juego, por esta razón se genera una columna nueva con el registro del año, si este existe, o añadiendo _Dato no disponible_, si la fecha no aparece. Por último, se suprime el campo **release_date**.


In [ ]:
# Cantidad de registros por cada fecha
Data_Games['release_date'].value_counts()

In [ ]:
# Creación de una columna nueva con el registro del año
Data_Games['Year_Release'] = Data_Games['release_date'].apply(FuncionesDA.Year_release)

# Eliminación del campo 'release_date'
Data_Games = Data_Games.drop('release_date', axis=1)
Data_Games.head()

In [ ]:
Data_Games['Year_Release'].unique()

5. Transformación de la columna **price**.

Este campo es de utilidad para el proyecto, pero existen textos en casos de promociones o en caso de que el juego es gratuito. En caso de ser gratis, se reemplazarán esos registros por 0. Además a los valores nulos se les asignará también el valor cero.


In [ ]:
Data_Games['price'] = Data_Games['price'].apply(FuncionesDA.Float_Value)
Data_Games['price'].dtype

6. Transformación de las variables **publisher**, **app_name**, **title**, **developer.**

Estas columnas presentan valores nulos, por lo que se completarán con 'Dato no disponible' para que el tipo de dato sea el mismo en cada una de esas columnas.

In [ ]:
# Variables a transformar
columnas_a_completar = ['publisher', 'app_name', 'title', 'developer']

# Se rellenan los nulos
Data_Fill = Data_Games[columnas_a_completar].fillna('Dato no disponible')

# Borrado de las variables originales y se concatenan las rellenas con todo el dataframe
Data_Games = pd.concat([Data_Games.drop(columnas_a_completar, axis=1), Data_Fill], axis=1)
Data_Games.head()

7. Transformación de la variable **genres.**

La columna **genres** esta constituida por una lista que contiene los distintos géneros de los videojuegos. Es necesario construir una nueva columna con cada uno de los géneros.

In [ ]:
Data_Games = Data_Games.explode('genres')
Data_Games = Data_Games.dropna(subset=['genres'])
Data_Games.head()

8. Eliminación de las variables **tags**, **specs**, **url**, **reviews_url.**

Al ser variables sin utilidad para la implementación del proyecto, se eliminarán las respectivas columnas del DataFrame.

In [ ]:
Data_Games = Data_Games.drop(['tags', 'specs', 'url', 'reviews_url'], axis=1)
Data_Games.columns

9. Verificación final.

Finalmente, se verifican el tipo de dato de cada variable del DataFrame y si aún existen observaciones _NaN_.

In [ ]:
# Verificación de el tipo de datos por columna existencia  de valores nulos
FuncionesDA.Data_Type(Data_Games)

10. Carga del conjunto de datos `output_steam_games.json`

Se guarda el conjunto de datos transformado por el poceso de ***ETL*** como `Steam_games`.

In [ ]:
# Opcion 01 --> Formato Parquet
filepath = 'Data/Datasets/Parquet/Steam_games.parquet'
Data_Games.to_parquet(filepath, engine='pyarrow')
print(f'Se guardó el archivo {filepath}')


In [ ]:
# Opcion 02 --> Formato csv
Filepath = 'Data/Datasets/csv/Steam_games.csv'
Data_Games.to_csv(Filepath, index=False, encoding='utf-8')
print(f'Se guardó el archivo {Filepath}')

<br> <br> <br> <br><br> <br> <br> <br>
***
**1.2.2.  Archivo `australian_users_items.json`**

Lectura del archivo en formato _json_ y creación de un DataFrame para el inicio del proceso de ***ETL***.

In [ ]:
# Ruta al dataset australian_user_reviews
filepath_items = "Data/Raw Data/australian_users_items.json"

# Se lee de cada línea del conjunto de datos original
rows_items = []
with open(filepath_items, 'r', encoding='utf-8') as file:
    for line in file.readlines():
        rows_items.append(ast.literal_eval(line))

# Se convierte en dataframe
Data_Items = pd.DataFrame(rows_items)


In [ ]:
Data_Items

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."
...,...,...,...,...,...
88305,76561198323066619,22,76561198323066619,http://steamcommunity.com/profiles/76561198323...,"[{'item_id': '413850', 'item_name': 'CS:GO Pla..."
88306,76561198326700687,177,76561198326700687,http://steamcommunity.com/profiles/76561198326...,"[{'item_id': '11020', 'item_name': 'TrackMania..."
88307,XxLaughingJackClown77xX,0,76561198328759259,http://steamcommunity.com/id/XxLaughingJackClo...,[]
88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"[{'item_id': '304930', 'item_name': 'Unturned'..."


Se revisa el tipo de dato de cada columna y si hay nulos.

In [ ]:
# Revisión del  tipo de datos de cada variable y existencia de valores nulos (NaN)
FuncionesDA.Data_Type(Data_Items)

,Variable,Type,NaN,No_NaN,NaN_(%),No_NaN_(%)
0,user_id,[<class 'str'>],0,88310,0.0,100.0
1,items_count,[<class 'int'>],0,88310,0.0,100.0
2,steam_id,[<class 'str'>],0,88310,0.0,100.0
3,user_url,[<class 'str'>],0,88310,0.0,100.0
4,items,[<class 'list'>],0,88310,0.0,100.0


La columna **items** contiene diccionarios como observaciones, así que se explorará la variable misma para conocer su estructura tomando uno de sus registros.

In [ ]:

Data_Items['items'][1]

[{'item_id': '10',
  'item_name': 'Counter-Strike',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '80',
  'item_name': 'Counter-Strike: Condition Zero',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '100',
  'item_name': 'Counter-Strike: Condition Zero Deleted Scenes',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '300',
  'item_name': 'Day of Defeat: Source',
  'playtime_forever': 220,
  'playtime_2weeks': 0},
 {'item_id': '30',
  'item_name': 'Day of Defeat',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '40',
  'item_name': 'Deathmatch Classic',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '60',
  'item_name': 'Ricochet',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '240',
  'item_name': 'Counter-Strike: Source',
  'playtime_forever': 62,
  'playtime_2weeks': 0},
 {'item_id': '280',
  'item_name': 'Half-Life: Source',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': 

El conjunto de datos en `australian_users_items.json` consta de 88310 filas y 5 columnas, sin valores nulos. Las variables son las siguientes:

* **user_id**: Identificador único del usuario.
* **items_count**: Cantidad de juegos que ha consumido el usuario. Es un número entero.
* **steam_id**: Número único de identificación para la plataforma.
* **user_url**: La _url_ del perfil del usuario.
* **items**: Lista de uno o mas diccionarios de los juegos que consume cada usuario. Cada diccionario tiene las siguientes claves:
  * **item_id**: Identificador del juego.
  * **item_name**: Nombre del del juego.
  * **playtime_forever**: Tiempo acumulado que un usuario consumió un juego.
  * **playtime_2weeks**: Tiempo acumulado que un usuario jugó a un juego en las últimas dos semanas.

1. Transformación de la variable **items.**

La columna **items** se encuentra anidada, pues es una lista que contiene diccionarios. A continuación, se normaliza la columna de manera de obtener una columna por cada clave de ese diccionario, manteniendo la trazabilidad de las otras variables incluidas **steam_id**, **items_count**, **user_id** y **user_url**.

In [ ]:
# Normalización de la columna 'items'
Data_Items2 = pd.json_normalize(rows_items, record_path=['items'], meta=['steam_id','items_count','user_id', 'user_url'] )
Data_Items2

,item_id,item_name,playtime_forever,playtime_2weeks,steam_id,items_count,user_id,user_url
0,10,Counter-Strike,6,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
1,20,Team Fortress Classic,0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
2,30,Day of Defeat,7,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
3,40,Deathmatch Classic,0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
4,50,Half-Life: Opposing Force,0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
...,...,...,...,...,...,...,...,...
5153204,346330,BrainBread 2,0,0,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...
5153205,373330,All Is Dust,0,0,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...
5153206,388490,One Way To Die: Steam Edition,3,3,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...
5153207,521570,You Have 10 Seconds 2,4,4,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...


Se verifican los tipos de datos y si hay nulos.

In [ ]:
FuncionesDA.Data_Type(Data_Items2)

,Variable,Type,NaN,No_NaN,NaN_(%),No_NaN_(%)
0,item_id,[<class 'str'>],0,5153209,0.0,100.0
1,item_name,[<class 'str'>],0,5153209,0.0,100.0
2,playtime_forever,[<class 'int'>],0,5153209,0.0,100.0
3,playtime_2weeks,[<class 'int'>],0,5153209,0.0,100.0
4,steam_id,[<class 'str'>],0,5153209,0.0,100.0
5,items_count,[<class 'int'>],0,5153209,0.0,100.0
6,user_id,[<class 'str'>],0,5153209,0.0,100.0
7,user_url,[<class 'str'>],0,5153209,0.0,100.0


Se revisan si hay observaciones duplicadas.

In [ ]:
# Devuelve los duplicados
Duplicados = Data_Items2.loc[Data_Items2.duplicated()]
Duplicados

,item_id,item_name,playtime_forever,playtime_2weeks,steam_id,items_count,user_id,user_url
164294,20,Team Fortress Classic,5,0,76561198084006094,109,Nikiad,http://steamcommunity.com/id/Nikiad
164295,50,Half-Life: Opposing Force,0,0,76561198084006094,109,Nikiad,http://steamcommunity.com/id/Nikiad
164296,70,Half-Life,0,0,76561198084006094,109,Nikiad,http://steamcommunity.com/id/Nikiad
164297,130,Half-Life: Blue Shift,0,0,76561198084006094,109,Nikiad,http://steamcommunity.com/id/Nikiad
164298,220,Half-Life 2,198,0,76561198084006094,109,Nikiad,http://steamcommunity.com/id/Nikiad
...,...,...,...,...,...,...,...,...
4898223,213670,South Park™: The Stick of Truth™,725,0,76561198080057659,39,76561198080057659,http://steamcommunity.com/profiles/76561198080...
4898224,221910,The Stanley Parable,53,0,76561198080057659,39,76561198080057659,http://steamcommunity.com/profiles/76561198080...
4898225,261030,The Walking Dead: Season Two,253,0,76561198080057659,39,76561198080057659,http://steamcommunity.com/profiles/76561198080...
4898226,273110,Counter-Strike Nexon: Zombies,0,0,76561198080057659,39,76561198080057659,http://steamcommunity.com/profiles/76561198080...


Existen 59104 observaciones duplicadas, por lo que pueden ser eliminados.

In [ ]:
Data_Items2 = Data_Items2.drop_duplicates(keep='first')
Data_Items2

,item_id,item_name,playtime_forever,playtime_2weeks,steam_id,items_count,user_id,user_url
0,10,Counter-Strike,6,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
1,20,Team Fortress Classic,0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
2,30,Day of Defeat,7,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
3,40,Deathmatch Classic,0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
4,50,Half-Life: Opposing Force,0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
...,...,...,...,...,...,...,...,...
5153204,346330,BrainBread 2,0,0,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...
5153205,373330,All Is Dust,0,0,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...
5153206,388490,One Way To Die: Steam Edition,3,3,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...
5153207,521570,You Have 10 Seconds 2,4,4,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...


La columna **playtime_2weeks** puede ser eliminada, pues la información contenida en ella es acerca del tiempo acumulado de juego por cada usuario, que también aparece en **playtime_forever**.

In [ ]:
Data_Items2 = Data_Items2.drop('playtime_2weeks', axis=1)
Data_Items2.columns

Index(['item_id', 'item_name', 'playtime_forever', 'steam_id', 'items_count',
       'user_id', 'user_url'],
      dtype='object')

2. Carga del conjunto de datos `australian_users_items.json`

Se guarda el conjunto de datos transformados por el poceso de ***ETL*** como `User_items`.

In [ ]:
# Opcion 01 --> Formato Parquet
filepath01 = "Data/Datasets/Parquet/User_items.parquet"
Data_Items2.to_parquet(filepath01, engine='pyarrow')
print(f'Se guardó el archivo {filepath01}')


In [ ]:
Filepath01 = "Data/Datasets/csv/User_items.csv"
Data_Items2.to_csv(Filepath01, index=False, encoding='utf-8')
print(f'Se guardó el archivo {Filepath01}')

<br> <br> <br> <br><br> <br> <br> <br>
***

**1.2.3.  Archivo `australian_users_reviews.json`**

Lectura del archivo en formato _json_ y creación de un DataFrame para el inicio del proceso de ***ETL*** (Extracción, Tranformación y Carga). Se lee el conjunto de datos originales que contiene las reseñas hechas por los usuarios de ***Steam*** y se guarda en un archivo en formato DataFrame.

In [ ]:
# Ruta al conjunto de datos australian_user_reviews
filepath_reviews = "Data/Raw Data/australian_user_reviews.json"

# Lectura línea por linea del conjunto de datos original
rows_reviews = []
with open(filepath_reviews, 'r', encoding='utf-8') as file:
    for line in file.readlines():
        rows_reviews.append(ast.literal_eval(line))

# Conversión a objeto tipo DataFrame
Data_Reviews = pd.DataFrame(rows_reviews)

Data_Reviews

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."
...,...,...,...
25794,76561198306599751,http://steamcommunity.com/profiles/76561198306...,"[{'funny': '', 'posted': 'Posted May 31.', 'la..."
25795,Ghoustik,http://steamcommunity.com/id/Ghoustik,"[{'funny': '', 'posted': 'Posted June 17.', 'l..."
25796,76561198310819422,http://steamcommunity.com/profiles/76561198310...,"[{'funny': '1 person found this review funny',..."
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"[{'funny': '', 'posted': 'Posted July 21.', 'l..."


Se procede a verificar el tipo de dato por cada variable y el conteo de observaciones nulas.

In [ ]:
FuncionesDA.Data_Type(Data_Reviews)

,Variable,Type,NaN,No_NaN,NaN_(%),No_NaN_(%)
0,user_id,[<class 'str'>],0,25799,0.0,100.0
1,user_url,[<class 'str'>],0,25799,0.0,100.0
2,reviews,[<class 'list'>],0,25799,0.0,100.0


Para la variable **user_id** se verificará la existencia de observaciones duplicadas.

In [ ]:
row_dup = FuncionesDA.Duplicates(Data_Reviews, 'user_id')
row_dup

,user_id,user_url,reviews
12888,05041129,http://steamcommunity.com/id/05041129,"[{'funny': '', 'posted': 'Posted May 18, 2015...."
5250,05041129,http://steamcommunity.com/id/05041129,"[{'funny': '', 'posted': 'Posted May 18, 2015...."
3133,111222333444555666888,http://steamcommunity.com/id/11122233344455566...,"[{'funny': '', 'posted': 'Posted December 22, ..."
3134,111222333444555666888,http://steamcommunity.com/id/11122233344455566...,"[{'funny': '', 'posted': 'Posted December 22, ..."
4139,29123,http://steamcommunity.com/id/29123,"[{'funny': '', 'posted': 'Posted March 26.', '..."
...,...,...,...
2721,xXAussieRockXx,http://steamcommunity.com/id/xXAussieRockXx,"[{'funny': '', 'posted': 'Posted July 17, 2015..."
2680,yolofaceguy,http://steamcommunity.com/id/yolofaceguy,"[{'funny': '', 'posted': 'Posted October 31, 2..."
17916,yolofaceguy,http://steamcommunity.com/id/yolofaceguy,"[{'funny': '', 'posted': 'Posted October 31, 2..."
5855,zeroblade,http://steamcommunity.com/id/zeroblade,"[{'funny': '', 'posted': 'Posted November 30, ..."


Se observan 623 filas duplicadas en el campo **user_id**, pero se revisan si las reseñas dentro de los datos anidados de **review** también se encuentran duplicadas o solamente se repite el identificador del usuario (puede haya más de un comentario realizado por el mismo usuario).

In [ ]:
# Se revisa un identificador de usuario de ejemplo
user_id = '29123'
user_reviews = row_dup[row_dup['user_id'] == user_id]['reviews']

for review_list in user_reviews:
    for review in review_list:
        print(review['review'])
    print('-' * 40)

Can't play after the updates, people who doesn't know how to play the game, after playing the game crashes and The ♥♥♥♥ing black screen
5/10 lots of bugs and bad servers.BTW this game needs a better training facility
it needs blood when get hit, and some others changes, 9/10
What can i say? Nice graphics and nice story and charactersThe only thing wrong: it should be playable for all graphics cards even the lowest ones like 12 fps or 6But nvm nice game 8/10
Well, this game is the best i ever played but when i was downloading this game (again) i was like in 30%, but i connected today and it was 0% well if valve can fix this things i will be more happy and i will play more this game.
----------------------------------------
Can't play after the updates, people who doesn't know how to play the game, after playing the game crashes and The ♥♥♥♥ing black screen
5/10 lots of bugs and bad servers.BTW this game needs a better training facility
it needs blood when get hit, and some others change

Se observa que las reseñas  son exactamente las mismas para cada registro, por lo que se decide borrar los duplicados, dejando la primer ocurrencia de los registros.

In [ ]:
Data_Reviews = Data_Reviews.drop_duplicates(subset='user_id', keep='first')
FuncionesDA.Duplicates(Data_Reviews, 'user_id')

'No existen valores duplicados'

La columna **reviews** se revisa para conocer el tipo de dato que contiene.

In [ ]:
# Se observa el tipo de dato que contiene la columna'reviews'
Data_Reviews['reviews'][1]

[{'funny': '',
  'posted': 'Posted June 24, 2014.',
  'last_edited': '',
  'item_id': '251610',
  'helpful': '15 of 20 people (75%) found this review helpful',
  'recommend': True,
  'review': 'I know what you think when you see this title "Barbie Dreamhouse Party" but do not be intimidated by it\'s title, this is easily one of my GOTYs. You don\'t get any of that cliche game mechanics that all the latest games have, this is simply good core gameplay. Yes, you can\'t 360 noscope your friends, but what you can do is show them up with your bad ♥♥♥ dance moves and put them to shame as you show them what true fashion and color combinations are.I know this game says for kids but, this is easily for any age range and any age will have a blast playing this.8/8'},
 {'funny': '',
  'posted': 'Posted September 8, 2013.',
  'last_edited': '',
  'item_id': '227300',
  'helpful': '0 of 1 people (0%) found this review helpful',
  'recommend': True,
  'review': "For a simple (it's actually not all th

In [ ]:
Data_Reviews['reviews'].isnull()

0        False
1        False
2        False
3        False
4        False
         ...  
25794    False
25795    False
25796    False
25797    False
25798    False
Name: reviews, Length: 25485, dtype: bool

In [ ]:
Data_Reviews['reviews'].size

25485

Este campo contiene 25485 filas y 3 columnas, sin valores nulos. Las columnas son las siguientes:

* **user_id**: Identificador del usuario.
* **user_url**: La _url_ del perfil del usuario en _Streamcommunity_.
* **reviews**: Lista de diccionarios por usuario. Cada diccionario contiene:
    * **funny**: Indica si alguien puso emoticón de gracioso a la reseña del juego.
    * **posted**: Fecha publicación de la reseña. Un ejemplo del formato es _Posted April 21, 2011_.
    * **last_edited**: Fecha de la última edición.
    * **item_id**: Identificador único del juego.
    * **helpful**: Indica si fue útil la información.
    * **recommend**: Valor booleano que indica si el usuario recomienda o no el juego.
    * **review**: Comentarios sobre el juego.

1. Transformación de la variable **reviews.**

Esta variable presenta información anidada en diccionarios. Se busca generar un columna por cada diccionario para posteriormente hacer un registro por cada uno de ellos.

In [ ]:
# Se transforma a columnas cada elemento de las listas
Data_Reviews2 = pd.json_normalize(Data_Reviews['reviews'])
Data_Reviews2.head()

,0,1,2,3,4,5,6,7,8,9
0,"{'funny': '', 'posted': 'Posted November 5, 20...","{'funny': '', 'posted': 'Posted July 15, 2011....","{'funny': '', 'posted': 'Posted April 21, 2011...",None,None,None,None,None,None,None
1,"{'funny': '', 'posted': 'Posted June 24, 2014....","{'funny': '', 'posted': 'Posted September 8, 2...","{'funny': '', 'posted': 'Posted November 29, 2...",None,None,None,None,None,None,None
2,"{'funny': '', 'posted': 'Posted February 3.', ...","{'funny': '', 'posted': 'Posted December 4, 20...","{'funny': '', 'posted': 'Posted November 3, 20...","{'funny': '', 'posted': 'Posted October 15, 20...","{'funny': '', 'posted': 'Posted October 15, 20...","{'funny': '', 'posted': 'Posted October 15, 20...",None,None,None,None
3,"{'funny': '', 'posted': 'Posted October 14, 20...","{'funny': '', 'posted': 'Posted July 28, 2012....","{'funny': '', 'posted': 'Posted June 2, 2012.'...","{'funny': '', 'posted': 'Posted June 29, 2014....","{'funny': '', 'posted': 'Posted November 22, 2...","{'funny': '', 'posted': 'Posted February 23, 2...",None,None,None,None
4,"{'funny': '3 people found this review funny', ...","{'funny': '1 person found this review funny', ...","{'funny': '2 people found this review funny', ...","{'funny': '', 'posted': 'Posted July 11, 2013....",None,None,None,None,None,None


Conn esta transformación, se pierde el identificador del usuario (**user_id**) y la dirección web del usuario (**user_url**) al que pertenece cada diccionario, pero aún mantiene la misma posición. Para volver a tener la trazabilidad del usuario, se concatena con el DataFrame anterior.

In [ ]:
# Se agrega el número de identificación del usuario ('user_id') y el 'user_url' a las columnas separadas
Data_Reviews2 = pd.concat([Data_Reviews[['user_id', 'user_url']], Data_Reviews2], axis=1)
Data_Reviews2.head()

,user_id,user_url,0,1,2,3,4,5,6,7,8,9
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted November 5, 20...","{'funny': '', 'posted': 'Posted July 15, 2011....","{'funny': '', 'posted': 'Posted April 21, 2011...",None,None,None,None,None,None,None
1,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted June 24, 2014....","{'funny': '', 'posted': 'Posted September 8, 2...","{'funny': '', 'posted': 'Posted November 29, 2...",None,None,None,None,None,None,None
2,evcentric,http://steamcommunity.com/id/evcentric,"{'funny': '', 'posted': 'Posted February 3.', ...","{'funny': '', 'posted': 'Posted December 4, 20...","{'funny': '', 'posted': 'Posted November 3, 20...","{'funny': '', 'posted': 'Posted October 15, 20...","{'funny': '', 'posted': 'Posted October 15, 20...","{'funny': '', 'posted': 'Posted October 15, 20...",None,None,None,None
3,doctr,http://steamcommunity.com/id/doctr,"{'funny': '', 'posted': 'Posted October 14, 20...","{'funny': '', 'posted': 'Posted July 28, 2012....","{'funny': '', 'posted': 'Posted June 2, 2012.'...","{'funny': '', 'posted': 'Posted June 29, 2014....","{'funny': '', 'posted': 'Posted November 22, 2...","{'funny': '', 'posted': 'Posted February 23, 2...",None,None,None,None
4,maplemage,http://steamcommunity.com/id/maplemage,"{'funny': '3 people found this review funny', ...","{'funny': '1 person found this review funny', ...","{'funny': '2 people found this review funny', ...","{'funny': '', 'posted': 'Posted July 11, 2013....",None,None,None,None,None,None


Ahora que se tienen los diccionarios por columnas, con el usuario que genera dicha información, se genera un registro por cada diccionario, manteniendo en cada caso el usuario que lo genera.

In [ ]:
# Se utiliza pd.melt para transformar las columnas en filas conservando el 'user_id' y 'user_url'
Data_Reviews2 = pd.melt(Data_Reviews2, id_vars=['user_id', 'user_url'],
                       value_vars=list(range(9)),
                       value_name='reviews')
Data_Reviews2.head()

,user_id,user_url,variable,reviews
0,diego9031,http://steamcommunity.com/id/diego9031,0,"{'funny': '', 'posted': 'Posted July 30.', 'la..."
1,2768820078,http://steamcommunity.com/id/2768820078,0,"{'funny': '4 people found this review funny', ..."
2,tarjla,http://steamcommunity.com/id/tarjla,0,"{'funny': '34 people found this review funny',..."
3,jjas01,http://steamcommunity.com/id/jjas01,0,"{'funny': '', 'posted': 'Posted November 3, 20..."
4,76561197974500703,http://steamcommunity.com/profiles/76561197974...,0,"{'funny': '', 'posted': 'Posted July 15, 2012...."


Al hacer esto último se puede ver que quedan registros nulos. Esto ocurre porque hay usuarios que hicieron mas reseñas que otros. En este ejemplo se puede ver este caso:

In [ ]:
Data_Reviews2[Data_Reviews2['user_id']=='76561197970982479']

,user_id,user_url,0,1,2,3,4,5,6,7,8,9
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted November 5, 20...","{'funny': '', 'posted': 'Posted July 15, 2011....","{'funny': '', 'posted': 'Posted April 21, 2011...",None,None,None,None,None,None,None


Se procede a la eliminación los registros que contienen _None_ en la columna **reviews.**

In [ ]:
# Eliminación de las filas con valor None
Data_Reviews2 = Data_Reviews2.dropna()
# Se verifica que solo queden el identificador del usuario con la cantidad de diccionarios
# que le corresponde (se muestra un ejemplo)
Data_Reviews2[Data_Reviews2['user_id']=='76561197970982479']

,user_id,user_url,variable,reviews


En este punto ya es posible convertir cada diccionario en columna.

In [ ]:
# Se separan por columnas cada una de las claves del diccionario en 'reviews'
Data_Reviews = Data_Reviews2['reviews'].apply(pd.Series, dtype='object')
Data_Reviews = Data_Reviews.add_prefix('reviews_')
Data_Reviews.head()

,reviews_funny,reviews_posted,reviews_last_edited,reviews_item_id,reviews_helpful,reviews_recommend,reviews_review
0,,Posted July 30.,,4000,0 of 1 people (0%) found this review helpful,True,Vale a pena a pagar 20 R$ nesse jogo porque: ...
1,4 people found this review funny,"Posted September 1, 2015.",,287700,4 of 5 people (80%) found this review helpful,True,一部小岛秀夫的游戏，游戏中居然还出现了这句话，算是konami对玩家的妥协么
2,34 people found this review funny,Posted August 26.,,242760,52 of 62 people (84%) found this review helpful,True,So... You crashed your plane. You absolute ♥♥♥...
3,,"Posted November 3, 2014.",,208812,16 of 20 people (80%) found this review helpful,False,The hardest thing about this DLC is overcoming...
4,,"Posted July 15, 2012.",,8980,No ratings yet,True,An FPS that is somewhat Rogue-like. Some glari...


En el procesamiento anterior, se puede ver que la columna de **user_id** y **user_url** se perdió nuevamente, por lo que se vuelve a concatenar.

In [ ]:
# Se une con el 'user_id' y 'user_url'
Data_Reviews = pd.concat([Data_Reviews2[['user_id', 'user_url']], Data_Reviews], axis=1)
Data_Reviews.head()

,user_id,user_url,reviews_funny,reviews_posted,reviews_last_edited,reviews_item_id,reviews_helpful,reviews_recommend,reviews_review
0,diego9031,http://steamcommunity.com/id/diego9031,,Posted July 30.,,4000,0 of 1 people (0%) found this review helpful,True,Vale a pena a pagar 20 R$ nesse jogo porque: ...
1,2768820078,http://steamcommunity.com/id/2768820078,4 people found this review funny,"Posted September 1, 2015.",,287700,4 of 5 people (80%) found this review helpful,True,一部小岛秀夫的游戏，游戏中居然还出现了这句话，算是konami对玩家的妥协么
2,tarjla,http://steamcommunity.com/id/tarjla,34 people found this review funny,Posted August 26.,,242760,52 of 62 people (84%) found this review helpful,True,So... You crashed your plane. You absolute ♥♥♥...
3,jjas01,http://steamcommunity.com/id/jjas01,,"Posted November 3, 2014.",,208812,16 of 20 people (80%) found this review helpful,False,The hardest thing about this DLC is overcoming...
4,76561197974500703,http://steamcommunity.com/profiles/76561197974...,,"Posted July 15, 2012.",,8980,No ratings yet,True,An FPS that is somewhat Rogue-like. Some glari...


Se observa que hay valores faltantes en algunas columnas, pero no estan como nulos, probablemente deben tener un espacio. Se compueba esto.

In [ ]:
Data_Reviews['reviews_last_edited'][0]

''

Se reemplazan esos espacios como valores nulos.

In [ ]:
Data_Reviews.replace('', None, inplace=True)
Data_Reviews.head()

,user_id,user_url,reviews_funny,reviews_posted,reviews_last_edited,reviews_item_id,reviews_helpful,reviews_recommend,reviews_review
0,diego9031,http://steamcommunity.com/id/diego9031,None,Posted July 30.,None,4000,0 of 1 people (0%) found this review helpful,True,Vale a pena a pagar 20 R$ nesse jogo porque: ...
1,2768820078,http://steamcommunity.com/id/2768820078,4 people found this review funny,"Posted September 1, 2015.",None,287700,4 of 5 people (80%) found this review helpful,True,一部小岛秀夫的游戏，游戏中居然还出现了这句话，算是konami对玩家的妥协么
2,tarjla,http://steamcommunity.com/id/tarjla,34 people found this review funny,Posted August 26.,None,242760,52 of 62 people (84%) found this review helpful,True,So... You crashed your plane. You absolute ♥♥♥...
3,jjas01,http://steamcommunity.com/id/jjas01,None,"Posted November 3, 2014.",None,208812,16 of 20 people (80%) found this review helpful,False,The hardest thing about this DLC is overcoming...
4,76561197974500703,http://steamcommunity.com/profiles/76561197974...,None,"Posted July 15, 2012.",None,8980,No ratings yet,True,An FPS that is somewhat Rogue-like. Some glari...


Se analizan los tipos de datos y los nulos que quedaron luego de desanidar la columna **reviews**.

In [ ]:
FuncionesDA.Data_Type(Data_Reviews)

,Variable,Type,NaN,No_NaN,NaN_(%),No_NaN_(%)
0,user_id,[<class 'str'>],0,2322,0.000,100.000
1,user_url,[<class 'str'>],0,2322,0.000,100.000
2,reviews_funny,"[<class 'NoneType'>, <class 'str'>]",1861,461,80.146,19.854
3,reviews_posted,[<class 'str'>],0,2322,0.000,100.000
4,reviews_last_edited,"[<class 'NoneType'>, <class 'str'>]",1965,357,84.625,15.375
5,reviews_item_id,[<class 'str'>],0,2322,0.000,100.000
6,reviews_helpful,[<class 'str'>],0,2322,0.000,100.000
7,reviews_recommend,[<class 'bool'>],0,2322,0.000,100.000
8,reviews_review,[<class 'str'>],0,2322,0.000,100.000


Se observa que **reviews_funny** presenta un 80% y **reviews_last_edited** tiene un 84.6% de observaciones _NaN_, por lo que se decide eliminarlas.

In [ ]:
# Eliminación de las columnas 'reviews_funny' y 'reviews_last_edited'
Data_Reviews = Data_Reviews.drop(columns=['reviews_funny', 'reviews_last_edited'])

Data_Reviews.columns

Index(['user_id', 'user_url', 'reviews_posted', 'reviews_item_id',
       'reviews_helpful', 'reviews_recommend', 'reviews_review'],
      dtype='object')

2. Transformación de la columna **reviews_posted.**

Es necesario que la fecha donde se hizo la publicación de la reseña esté en un único formato `YYYY-MM-DD`. Por lo tanto, es necesario procesar la fecha y extraer los elementos relevantes. Se utilizará expresiones regulares para buscar y capturar los valores de año, mes y día dentro de la cadena de texto.

In [ ]:
Data_Reviews['reviews_date'] = Data_Reviews['reviews_posted'].apply(FuncionesDA.Date_Convert)

Data_Reviews['reviews_date']

0       Formato inválido
1             2015-09-01
2       Formato inválido
3             2014-11-03
4             2012-07-15
              ...       
2317          2015-07-14
2318          2014-10-21
2319          2015-08-27
2320          2014-08-15
2321          2014-08-02
Name: reviews_date, Length: 2322, dtype: object

Dado que existen registros que contienen un formato inválido distinto a los demas registros. En este caso, no contiene el año del posteo, pero con la función se imputó como _Formato inválido_. Estos registros no se podrán consultar desde la ***API***, pero las demás columnas serán útiles para aportar información.

In [ ]:
Data_Reviews[Data_Reviews['reviews_date'] == 'Formato inválido']

,user_id,user_url,reviews_posted,reviews_item_id,reviews_helpful,reviews_recommend,reviews_review,reviews_date
0,diego9031,http://steamcommunity.com/id/diego9031,Posted July 30.,4000,0 of 1 people (0%) found this review helpful,True,Vale a pena a pagar 20 R$ nesse jogo porque: ...,Formato inválido
2,tarjla,http://steamcommunity.com/id/tarjla,Posted August 26.,242760,52 of 62 people (84%) found this review helpful,True,So... You crashed your plane. You absolute ♥♥♥...,Formato inválido
5,armouredmarshmallow,http://steamcommunity.com/id/armouredmarshmallow,Posted July 20.,33230,3 of 4 people (75%) found this review helpful,True,haven't played any other assassin's creed game...,Formato inválido
6,darkmassieh,http://steamcommunity.com/id/darkmassieh,Posted June 28.,47780,10 of 16 people (63%) found this review helpful,True,great game youll enjoy this game if A: you enj...,Formato inválido
11,harharharharhar,http://steamcommunity.com/id/harharharharhar,Posted June 30.,286160,2 of 3 people (67%) found this review helpful,True,Best community. Very nice people and are alway...,Formato inválido
...,...,...,...,...,...,...,...,...
2188,Shaumenka,http://steamcommunity.com/id/Shaumenka,Posted January 10.,50300,0 of 3 people (0%) found this review helpful,True,... what the ♥♥♥♥ did i just play!?,Formato inválido
2202,76561198085013601,http://steamcommunity.com/profiles/76561198085...,Posted May 15.,466910,No ratings yet,True,Muito viciante e divertido! :D,Formato inválido
2247,76561197961124706,http://steamcommunity.com/profiles/76561197961...,Posted March 22.,435870,8 of 49 people (16%) found this review helpful,False,kkk ♥♥♥♥,Formato inválido
2252,m0ng,http://steamcommunity.com/id/m0ng,Posted January 19.,363970,No ratings yet,True,Jogo legal pa carai meu dedo agora vira pro la...,Formato inválido


Se decide elimina la columna **reviews_posted**, dado que no aporta información relevante, ya que solamente muestra el día y el mes de publicación de la reseña.

In [ ]:
Data_Reviews = Data_Reviews.drop('reviews_posted', axis=1)
Data_Reviews.columns

Index(['user_id', 'user_url', 'reviews_item_id', 'reviews_helpful',
       'reviews_recommend', 'reviews_review', 'reviews_date'],
      dtype='object')

Por último, de realiza una revisión final de la transformación del conjunto de datos.

In [ ]:
FuncionesDA.Data_Type(Data_Reviews)

,Variable,Type,NaN,No_NaN,NaN_(%),No_NaN_(%)
0,user_id,[<class 'str'>],0,2322,0.0,100.0
1,user_url,[<class 'str'>],0,2322,0.0,100.0
2,reviews_item_id,[<class 'str'>],0,2322,0.0,100.0
3,reviews_helpful,[<class 'str'>],0,2322,0.0,100.0
4,reviews_recommend,[<class 'bool'>],0,2322,0.0,100.0
5,reviews_review,[<class 'str'>],0,2322,0.0,100.0
6,reviews_date,[<class 'str'>],0,2322,0.0,100.0


3. Carga del conjunto de datos `australian_users_reviews.json`

Se guarda el conjunto de datos transformados por el poceso de ***ETL*** como `User_reviews`.

In [ ]:
# Opcion 01 --> Formato Parquet
filepath02 = "Data/Datasets/Parquet/User_reviews.parquet"
Data_Reviews.to_parquet(filepath02, engine='pyarrow')
print(f'Se guardó el archivo {filepath02}')


In [ ]:
# Opcion 02 --> Formato csv
Filepath02 = "Data/Datasets/csv/User_reviews.csv"
Data_Reviews.to_csv(Filepath02, index=False, encoding='utf-8')
print(f'Se guardó el archivo {Filepath02}')